In [ ]:
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2
import torchvision.transforms.functional as cvF

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
def gaussian_noise(image_tensor: torch.Tensor, mean=0.0, std=0.1):
    noise = torch.randn(image_tensor.size(), device=device) * std + mean
    noisy_image = image_tensor + noise
    
    return torch.clamp(noisy_image, 0, 1)

In [ ]:
image = Image.open("img1.png")

image_tensor = transforms.Grayscale()(transforms.ToTensor()(image)[0:3]).to(device).unsqueeze(0)

In [ ]:
_, _, height, width = image_tensor.shape

y_pos = height // 2 + 50

cv_image = cv2.imread('img1.png')
cv_image = cv2.line(cv_image, (0, y_pos), (width, y_pos), (255, 0, 0), 2)

plt.imshow(cv_image)
plt.axis('off')
plt.show()

### Get Intensity Profile

In [ ]:
def intensity_profile(image_tensor: torch.Tensor, y_pos: int):
    line = image_tensor[0][0][y_pos]
    y = line.cpu().numpy()
    x = np.linspace(0, len(y)-1, len(y))

    d_kernel = torch.tensor([-1, 1], device=device, dtype=torch.float)
    dy = F.conv1d(image_tensor[0][0][y_pos].unsqueeze(0), d_kernel.unsqueeze(0).unsqueeze(0))
    dy = F.pad(dy, (0, 1), mode='constant', value=0)
    dy = dy.squeeze().cpu().numpy()

    return x, y, dy

In [ ]:
x, y, dy = intensity_profile(image_tensor, y_pos)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(x, y)
plt.title('Intensity')

plt.subplot(1, 2, 2)
plt.plot(x, dy, color='r')
plt.title('Intensity Derivative')

plt.show();

### Add Gaussian Noise and Smooth for Intensity Profile

In [ ]:
noisy_image_tensor = gaussian_noise(image_tensor)
x, y, dy = intensity_profile(noisy_image_tensor, y_pos)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(x, y)
plt.title('Intensity')

plt.subplot(1, 2, 2)
plt.plot(x, dy, color='r')
plt.title('Intensity Derivative')
plt.show();

In [ ]:
smoothed_noisy_image_tensor = cvF.gaussian_blur(noisy_image_tensor, kernel_size=5, sigma=2)
x, y, dy = intensity_profile(smoothed_noisy_image_tensor, y_pos)

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(x, y)
plt.title('Intensity')

plt.subplot(1, 2, 2)
plt.plot(x, dy, color='r')
plt.title('Intensity Derivative')
plt.show();

### Prewitt Edge Detection

In [ ]:
kernel_x = torch.tensor([[1, 0, -1],
                       [1, 0, -1],
                       [1, 0, -1]], dtype=torch.float, device=device)
kernel_y = torch.tensor([[1, 1, 1],
                       [0, 0, 0],
                       [-1, -1, -1]], dtype=torch.float, device=device)

edges_x = F.conv2d(image_tensor, kernel_x.view(1, 1, 3, 3))
edges_y = F.conv2d(image_tensor, kernel_y.view(1, 1, 3, 3))
edges = torch.sqrt(edges_x ** 2 + edges_y ** 2)

edges = F.threshold(edges, 0.3, 0)

edges_x = edges_x.squeeze().cpu().numpy()
edges_y = edges_y.squeeze().cpu().numpy()
edges = edges.squeeze().cpu().numpy()

plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(edges_x, cmap='gray')
plt.title('Horizontal Derivative')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(edges_y, cmap='gray')
plt.title('Vertical Derivative')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(edges, cmap='gray')
plt.title('Prewitt Edge Detection')
plt.axis('off')

plt.show()

### Sobel Edge Detection

In [ ]:
kernel_x = torch.tensor([[1, 0, -1],
                       [2, 0, -2],
                       [1, 0, -1]], dtype=torch.float, device=device)
kernel_y = torch.tensor([[1, 2, 1],
                       [0, 0, 0],
                       [-1, -2, -1]], dtype=torch.float, device=device)

edges_x = F.conv2d(image_tensor, kernel_x.view(1, 1, 3, 3))
edges_y = F.conv2d(image_tensor, kernel_y.view(1, 1, 3, 3))
edges = torch.sqrt(edges_x ** 2 + edges_y ** 2)

edges = F.threshold(edges, 0.3, 0)

edges_x = edges_x.squeeze().cpu().numpy()
edges_y = edges_y.squeeze().cpu().numpy()
edges = edges.squeeze().cpu().numpy()

plt.figure(figsize=(15, 5))

plt.subplot(1, 4, 1)
plt.imshow(image, cmap='gray')
plt.title('Original Image')
plt.axis('off')

plt.subplot(1, 4, 2)
plt.imshow(edges_x, cmap='gray')
plt.title('Horizontal Derivative')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.imshow(edges_y, cmap='gray')
plt.title('Vertical Derivative')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.imshow(edges, cmap='gray')
plt.title('Sobel Edge Detection')
plt.axis('off')

plt.show()

### Marr Hildreth Edge Detection

In [ ]:
def laplacian_of_gaussian(image_tensor: torch.Tensor, kernel_size: int, sigma: float) -> torch.Tensor:
    # Apply Gaussian smoothing
    smoothed_image_tensor = cvF.gaussian_blur(image_tensor, kernel_size, sigma)

    # Apply Laplacian of Gaussian
    x = torch.arange(kernel_size, dtype=torch.float, device=device) - kernel_size // 2
    y = torch.arange(kernel_size, dtype=torch.float, device=device) - kernel_size // 2
    x, y = torch.meshgrid(x, y)
    normalization = 1 / (2.0 * np.pi * sigma**2)
    log_kernel = normalization * ((x**2 + y**2 - 2.0 * sigma**2) / (sigma**4)) * torch.exp(-(x**2 + y**2) / (2.0 * sigma**2))
    log_kernel = log_kernel - log_kernel.mean()
    log_kernel = log_kernel.unsqueeze(0).unsqueeze(0)
    log_image_tensor = F.conv2d(smoothed_image_tensor, log_kernel, padding=kernel_size // 2)

    return log_image_tensor.squeeze(0)

def zero_crossing(log_image_tensor):
    zero_crossings = torch.zeros_like(log_image_tensor)
    zero_crossings[:, 1:-1, 1:-1] = (
        (log_image_tensor[:, 1:-1, 1:-1] * log_image_tensor[:, :-2, 1:-1] < 0) |
        (log_image_tensor[:, 1:-1, 1:-1] * log_image_tensor[:, 2:, 1:-1] < 0) |
        (log_image_tensor[:, 1:-1, 1:-1] * log_image_tensor[:, 1:-1, :-2] < 0) |
        (log_image_tensor[:, 1:-1, 1:-1] * log_image_tensor[:, 1:-1, 2:] < 0)
    ).float()

    return zero_crossings.squeeze()


In [ ]:
smoothed_image_tensor = cvF.gaussian_blur(image_tensor, kernel_size=5, sigma=1)
log_image_tensor = laplacian_of_gaussian(smoothed_image_tensor, 5, 1)
edges = zero_crossing(log_image_tensor)

log_image = log_image_tensor.squeeze(0).cpu().numpy()
edges = edges.squeeze(0).cpu().numpy()

plt.figure(figsize=(12, 4))

plt.subplot(1, 3, 1)
plt.title("Original Image")
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 3, 2)
plt.title("Image after LoG")
plt.imshow(log_image, cmap='gray')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.title("Marr Hildreth Edge Detection")
plt.imshow(edges, cmap='gray')
plt.axis('off')

plt.show()

### Canny Edge Detection

In [ ]:
def gradients(image_tensor):
    kernel_x = torch.tensor([[1, 0, -1],
                       [2, 0, -2],
                       [1, 0, -1]], dtype=torch.float, device=device)
    kernel_y = torch.tensor([[1, 2, 1],
                        [0, 0, 0],
                        [-1, -2, -1]], dtype=torch.float, device=device)

    edges_x = F.conv2d(image_tensor, kernel_x.view(1, 1, 3, 3))
    edges_y = F.conv2d(image_tensor, kernel_y.view(1, 1, 3, 3))
    
    edges = torch.sqrt(edges_x ** 2 + edges_y ** 2)
    theta = torch.atan2(edges_y, edges_x)

    return edges, theta

def non_maximum_suppression(edges, theta):
    theta = theta * 180.0 / np.pi
    theta[theta < 0] += 180
    
    Z = torch.zeros_like(edges, device=device, dtype=torch.float)
    angle = theta % 180
    
    mask_0 = ((angle >= 0) & (angle < 22.5)) | ((angle >= 157.5) & (angle < 180))
    mask_45 = (angle >= 22.5) & (angle < 67.5)
    mask_90 = (angle >= 67.5) & (angle < 112.5)
    mask_135 = (angle >= 112.5) & (angle < 157.5)

    P0 = edges[:, :, 1:-1, 2:] * mask_0[:, :, 1:-1, 1:-1] + edges[:, :, 1:-1, :-2] * mask_0[:, :, 1:-1, 1:-1]
    P45 = edges[:, :, 2:, :-2] * mask_45[:, :, 1:-1, 1:-1] + edges[:, :, :-2, 2:] * mask_45[:, :, 1:-1, 1:-1]
    P90 = edges[:, :, 2:, 1:-1] * mask_90[:, :, 1:-1, 1:-1] + edges[:, :, :-2, 1:-1] * mask_90[:, :, 1:-1, 1:-1]
    P135 = edges[:, :, :-2, :-2] * mask_135[:, :, 1:-1, 1:-1] + edges[:, :, 2:, 2:] * mask_135[:, :, 1:-1, 1:-1]

    Q = torch.max(torch.max(P0, P45), torch.max(P90, P135))

    Z[:, :, 1:-1, 1:-1] = edges[:, :, 1:-1, 1:-1] * (edges[:, :, 1:-1, 1:-1] >= Q).float()
    
    return Z

def threshold(image_tensor: torch.Tensor, low_threshold=0.05, high_threshold=0.15):
    high_threshold = image_tensor.max() * high_threshold
    low_threshold = high_threshold * low_threshold
    
    M, N = image_tensor.shape[2], image_tensor.shape[3]
    res = torch.zeros_like(image_tensor, device=device, dtype=torch.float)
    
    strong = 255
    weak = 25
    
    _, _, strong_i, strong_j = torch.where(image_tensor >= high_threshold)
    _, _, zeros_i, zeros_j = torch.where(image_tensor < low_threshold)
    
    _, _, weak_i, weak_j = torch.where((image_tensor <= high_threshold) & (image_tensor >= low_threshold))
    
    res[0][0][strong_i, strong_j] = strong
    res[0][0][weak_i, weak_j] = weak
    
    return res

def hysteresis(image_tensor: torch.Tensor):
    M, N = image_tensor.shape[2], image_tensor.shape[3]
    weak = 25
    strong = 255

    _, _, strong_i, strong_j = torch.where(image_tensor == strong)
    _, _, weak_i, weak_j = torch.where(image_tensor == weak)
    
    strong_pixels = set(zip(strong_i.tolist(), strong_j.tolist()))
    weak_pixels = set(zip(weak_i.tolist(), weak_j.tolist()))
    
    while strong_pixels:
        i, j = strong_pixels.pop()
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                ni, nj = i + di, j + dj
                if (di != 0 or dj != 0) and (0 <= ni < M) and (0 <= nj < N):
                    if (ni, nj) in weak_pixels:
                        image_tensor[0, 0, ni, nj] = strong
                        strong_pixels.add((ni, nj))
                        weak_pixels.remove((ni, nj))
    
    image_tensor[image_tensor == weak] = 0

    return image_tensor

In [ ]:
# Apply Gaussian blur to smooth image
smoothed_image_tensor = cvF.gaussian_blur(image_tensor, kernel_size=5, sigma=1)

# Compute gradients
edges, theta = gradients(image_tensor)

# Non-Maximum Suppression
non_maximum_suppressed_image_tensor = non_maximum_suppression(edges, theta)

# Thresholding
thresholded_image_tensor = threshold(non_maximum_suppressed_image_tensor)

# Edge Tracking with Hysteresis
edges = hysteresis(thresholded_image_tensor)

non_maximum_suppressed_image_tensor = non_maximum_suppressed_image_tensor.squeeze(0).squeeze(0).cpu().numpy()
thresholded_image_tensor = thresholded_image_tensor.squeeze(0).squeeze(0).cpu().numpy()
edges = edges.squeeze(0).squeeze(0).cpu().numpy()

plt.figure(figsize=(12, 4))

plt.subplot(1, 4, 1)
plt.title("Original Image")
plt.imshow(image)
plt.axis('off')

plt.subplot(1, 4, 2)
plt.title("Non-Maximum Suppression")
plt.imshow(non_maximum_suppressed_image_tensor, cmap='gray')
plt.axis('off')

plt.subplot(1, 4, 3)
plt.title("Thresholding")
plt.imshow(thresholded_image_tensor, cmap='gray')
plt.axis('off')

plt.subplot(1, 4, 4)
plt.title("Canny Edge Detection")
plt.imshow(edges, cmap='gray')
plt.axis('off')

plt.show()